SageMaker Studio notebooks provide a set of built-in images for popular data science and ML frameworks and compute options to run notebooks. The built-in SageMaker images contain the Amazon SageMaker Python SDK and the latest version of the backend runtime process, also called kernel. With the custom images feature, you can register custom built images and kernels, and make them available to all users sharing a SageMaker Studio domain. You can start by cloning and extending one of the example Docker files provided by SageMaker, or build your own images from scratch.


You can add a custom **R** image to SageMaker Studio so you can build and train your R models with SageMaker. After attaching the custom R image, you can select the image in Studio and use R to access the SDKs using the **RStudio reticulate** package.

You can create images and image versions and attach image versions to your domain using the SageMaker Studio Control Panel, the AWS SDK for Python (Boto3), and the AWS Command Line Interface (AWS CLI).

In [1]:
import sagemaker

ROLE_ARN = sagemaker.get_execution_role()

In [2]:
ROLE_ARN

'arn:aws:iam::367158743199:role/MySagemakerRole'

In [3]:
!aws sagemaker describe-notebook-instance --notebook-instance-name featurestore-streaming-agg | grep 'RoleArn:*'

    "RoleArn": "arn:aws:iam::367158743199:role/MySagemakerRole",


In [5]:
%%sh

ACCOUNT_ID=$(aws sts get-caller-identity --query Account --output text)
# Get the region defined in the current configuration (default to us-west-2 if none defined)
REGION=$(aws configure get region)
REGION=${REGION:-us-east-1}


# # create a repository in ECR, and then login to ECR repository
# aws --region ${REGION} ecr create-repository --repository-name smstudio-custom
# aws ecr --region ${REGION} get-login-password | docker login --username AWS \
#     --password-stdin ${ACCOUNT_ID}.dkr.ecr.${REGION}.amazonaws.com/smstudio-custom

# Build the docker image and push to Amazon ECR (modify image tags and name as required)
# $(aws ecr get-login --region ${REGION} --no-include-email)

# docker build . -t smstudio-r -t ${ACCOUNT_ID}.dkr.ecr.${REGION}.amazonaws.com/smstudio-custom:r
    
# docker push ${ACCOUNT_ID}.dkr.ecr.${REGION}.amazonaws.com/smstudio-custom:r
    
# Using with SageMaker Studio
## Create SageMaker Image with the image in ECR (modify image name as required)
# aws sagemaker create-image \
#     --region ${REGION} \
#     --image-name custom-r \
#     --role-arn "arn:aws:iam::367158743199:role/MySagemakerRol"


# aws sagemaker create-image-version \
#     --region ${REGION} \
#     --image-name custom-r \
#     --base-image ${ACCOUNT_ID}.dkr.ecr.${REGION}.amazonaws.com/smstudio-custom:r

## Create AppImageConfig for this image (modify AppImageConfigName and 
## KernelSpecs in app-image-config-input.json as needed)
## note that 'file://' is required in the file path
aws sagemaker create-app-image-config \
    --region ${REGION} \
    --cli-input-json file://app-image-config-input.json

{
    "AppImageConfigArn": "arn:aws:sagemaker:us-east-1:367158743199:app-image-config/custom-r-image-config"
}
